In [46]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import pandas as pd
import time
import lxml
import cchardet
import concurrent.futures
from multiprocessing import cpu_count,Pool

In [6]:
def get_links(price_diff,min_price, max_price):
    cr_id_list = []
    start_time=time.time()
    session_object = requests.Session()
    for i in range(((max_price-min_price)//price_diff)):
        upper_lim = min_price + (price_diff)*(i+1)
        lower_lim = min_price + (price_diff)*i
        print(f"upper: {upper_lim}" + f" lower: {lower_lim}")
        headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}
        website = f"https://www.arabam.com/ikinci-el/otomobil?currency=TL&maxPrice={upper_lim}&minPrice={lower_lim}&take=50&view=Detailed"
        response = session_object.get(website,headers=headers)
        soup = BeautifulSoup(response.text,'lxml')
        results = soup.find_all("span", {"class":"bold"})
        page_num = int(results[1].get_text())
        print(page_num)
        new_session = requests.Session()
        for i in range(page_num):
            website = f"https://www.arabam.com/ikinci-el/otomobil?currency=TL&maxPrice={upper_lim}&minPrice={lower_lim}&take=50&view=Detailed&page={i+1}"
            response = new_session.get(website,headers=headers)
            soup = BeautifulSoup(response.text,'lxml')
            results = soup.find_all("div", {"class":"content-container"})
            for elem in results[:-1:]:
                id = elem.find('p',{"class":"id"}).get_text().strip().split()[2]
                cr_id_list.append(f"https://www.arabam.com/ilan/{id}")
    end_time = time.time()
    print(print(end_time-start_time))
    return(cr_id_list)

In [7]:
car_links = get_links(10000,200000,300000)

upper: 210000 lower: 200000
49
upper: 220000 lower: 210000
50
upper: 230000 lower: 220000
50
upper: 240000 lower: 230000
50
upper: 250000 lower: 240000
50
upper: 260000 lower: 250000
50
upper: 270000 lower: 260000
50
upper: 280000 lower: 270000
48
upper: 290000 lower: 280000
48
upper: 300000 lower: 290000
48
442.740362405777
None


In [9]:
len(car_links)

24574

In [42]:

AllCarDF = pd.DataFrame()
global AllCarDF
def fetch(link):
    website = link
    headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}
    response = requests.get(website, headers=headers)
    soup = BeautifulSoup(response.text,'lxml')
    results = soup.find_all("li",{"class":"bcd-list-item"})
    if len(results) ==0:
        return
    results[-1].get_text().strip().split()
    car_dict = dict()
    for elem in results:  
        features_car = ''.join(elem.get_text().split()).split(":")
        car_dict.update({features_car[0]:features_car[1]})
    car_df = pd.DataFrame([car_dict])
    #AllCarDF = pd.concat([car_df,AllCarDF])
    
    print('.',end="",flush=True)
    

In [43]:
core_count = cpu_count()
core_count

8

In [48]:
if __name__ == '__main__':
    curr_session = requests.Session()
    headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}
    start_time = time.time()
    #==================>
    '''for link in car_links[:300:]:
        fetch(link,curr_session,headers)
    '''

    with Pool(core_count) as p:
        p.map(fetch, car_links[:300:])


    #==================>
    end_time = time.time()
    print(end_time-start_time)


Process SpawnPoolWorker-123:
Traceback (most recent call last):
  File "/Users/sonerkuyar/opt/anaconda3/envs/web/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/sonerkuyar/opt/anaconda3/envs/web/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/sonerkuyar/opt/anaconda3/envs/web/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/sonerkuyar/opt/anaconda3/envs/web/lib/python3.9/multiprocessing/queues.py", line 368, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'fetch' on <module '__main__' (built-in)>
Process SpawnPoolWorker-125:
Process SpawnPoolWorker-124:
Traceback (most recent call last):
  File "/Users/sonerkuyar/opt/anaconda3/envs/web/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Users/sonerkuyar/opt/anaconda3/envs/web/lib/python3.9/multiprocessi

KeyboardInterrupt: 